In [1]:
using BenchmarkTools
using DistributedArrays
struct Foo{T, N} d::DArray{T, N} end
struct Bar{T, N, A} d::DArray{T, N, A} end

In [2]:
d = drand(4, 3, 2)
l = localpart(d)
foo = Foo(d)
bar = Bar(d);

In [3]:
f(foobar, i, j, k) = localpart(foobar.d)[i, j, k]
g(d, i, j, k) = localpart(d)[i, j, k]

g (generic function with 1 method)

In [4]:
@btime f($foo, 3, 2, 1)

  43.232 ns (1 allocation: 16 bytes)


0.37718471311641577

In [5]:
@btime f($bar, 3, 2, 1)

  6.400 ns (0 allocations: 0 bytes)


0.37718471311641577

In [6]:
@btime g($d, 3, 2, 1)

  6.100 ns (0 allocations: 0 bytes)


0.37718471311641577

In [7]:
@btime getindex($l, 3, 2, 1)

  1.300 ns (0 allocations: 0 bytes)


0.37718471311641577

In [8]:
@code_warntype f(foo, 3, 2, 1)

MethodInstance for f(::Foo{Float64, 3}, ::Int64, ::Int64, ::Int64)
  from f(foobar, i, j, k) in Main at In[3]:1
Arguments
  #self#::Core.Const(f)
  foobar::Foo{Float64, 3}
  i::Int64
  j::Int64
  k::Int64
Body::Any
1 ─ %1 = Base.getproperty(foobar, :d)::DArray{Float64, 3}
│   %2 = Main.localpart(%1)::Any
│   %3 = Base.getindex(%2, i, j, k)::Any
└──      return %3



In [9]:
@code_warntype f(bar, 3, 2, 1)

MethodInstance for f(::Bar{Float64, 3, Array{Float64, 3}}, ::Int64, ::Int64, ::Int64)
  from f(foobar, i, j, k) in Main at In[3]:1
Arguments
  #self#::Core.Const(f)
  foobar::Bar{Float64, 3, Array{Float64, 3}}
  i::Int64
  j::Int64
  k::Int64
Body::Float64
1 ─ %1 = Base.getproperty(foobar, :d)::DArray{Float64, 3, Array{Float64, 3}}
│   %2 = Main.localpart(%1)::Array{Float64, 3}
│   %3 = Base.getindex(%2, i, j, k)::Float64
└──      return %3



In [10]:
isconcretetype(Array{Float64, 3})

true

In [11]:
isconcretetype(DArray{Float64, 3})

false

In [12]:
typeof(d)

DArray{Float64, 3, Array{Float64, 3}}

In [13]:
isconcretetype(DArray{Float64, 3, Array{Float64, 3}})

true